In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input
import tensorflow_model_optimization as tfmot
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
import tensorflow.keras.backend as K
import numpy as np
disable_eager_execution()
#enable_eager_execution()

In [5]:
img = np.zeros((1,224,224,3))
# input image dimensions
img_rows, img_cols = 224 ,224
input_shape = (img_rows, img_cols, 3)

In [3]:
K.clear_session()
# define input tensor as a placeholder
input_tensor = Input(shape=[img_rows, img_cols, 3],dtype=tf.float32 )
model = VGG16(input_tensor=input_tensor)

In [4]:
label = 1 
loss = model.get_layer('predictions').output[..., label] #prediction score for the first labeled class
grads = tf.gradients(loss, input_tensor)[0] #gradient wrt input
my_func1 = K.function([input_tensor], [loss,grads])

In [6]:
l,grads_value = my_func1(img)

In [7]:
grads 

<tf.Tensor 'gradients/block1_conv1/Conv2D_grad/Conv2DBackpropInput:0' shape=(None, 224, 224, 3) dtype=float32>

In [8]:
grads_value

array([[[[-3.0174573e-07, -1.9591373e-07, -6.2357168e-07],
         [ 6.3699542e-07,  8.3343809e-07,  6.7949131e-08],
         [ 6.3554955e-07,  7.5128139e-07,  1.0226256e-07],
         ...,
         [ 3.1516376e-07,  8.9249664e-07,  3.0592270e-07],
         [-1.9307819e-07,  5.0339560e-07,  4.2625466e-08],
         [-3.2156379e-07,  7.4459940e-08, -1.0066536e-07]],

        [[ 2.8785877e-07,  4.6422997e-07, -4.9973511e-07],
         [ 2.6554696e-06,  2.8442366e-06,  1.1804447e-06],
         [ 2.9480657e-06,  2.8692509e-06,  1.3597398e-06],
         ...,
         [ 1.4962332e-06,  2.3416783e-06,  1.2267468e-06],
         [ 1.3002781e-07,  1.1519543e-06,  3.2780386e-07],
         [-5.9840772e-07, -3.9379806e-08, -3.2764109e-07]],

        [[ 9.9312513e-07,  8.4483196e-07, -7.5169538e-07],
         [ 3.4999507e-06,  2.6276255e-06,  5.2366715e-09],
         [ 3.4566092e-06,  1.4672722e-06, -8.7326816e-07],
         ...,
         [ 2.0827724e-06,  1.8509977e-06,  8.5389843e-07],
         [

In [6]:
K.clear_session()
# define input tensor as a placeholder
input_tensor_2 = Input(shape=[img_rows, img_cols, 3],dtype=tf.float32 )
model_2 = VGG16(input_tensor=input_tensor_2)

# q_aware stands for for quantization aware.
q_model = tfmot.quantization.keras.quantize_model(model_2)

In [8]:
label = 1 
quantized_input = q_model.get_layer('quant_block1_conv1').input
loss2 = q_model.get_layer('quant_predictions').output[..., label] #prediction score for the first labeled class
grads = tf.gradients(loss2, quantized_input)[0] #gradient wrt input
my_func2  = K.function([input_tensor_2], [quantized_input,loss2,grads])

In [9]:
_,l,grads_value = my_func2(img)

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'input_1_5' with dtype float and shape [?,224,224,3]
	 [[{{node input_1_5}}]]
	 [[quant_predictions/cond_1/then/_892/MovingAvgQuantize/BatchMax/_2661]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'input_1_5' with dtype float and shape [?,224,224,3]
	 [[{{node input_1_5}}]]
0 successful operations.
0 derived errors ignored.